In [ ]:
# Copyright (c) 2012-2023, NECOTIS
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
#
#  - Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
#  - Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#  - Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
# IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT
# NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA,
# OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

# Authors: Emmanuel Calvet, Bertrand Reulet,  Jean Rouat (advisors)
# Date: November 30th, 2023
# Organization: Groupe de recherche en Neurosciences Computationnelles et Traitement Intelligent des Signaux (NECOTIS),
# Université de Sherbrooke, Canada


In [2]:
# Import library and models
# NB: you need to import binary_model_design into the python path
import sys
import os
import importlib

# Get the module path and add it to python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Setting the module 
model_path = os.path.join(module_path, 'model')
MODULE_PATH = f"{model_path}/__init__.py"
MODULE_NAME = "binary_model" # /!\ if you change this, you must also modify the call of the module in all dependant files!
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)
    
# Standard library
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# The model
from binary_model.binaryModel import binaryModel
from binary_model.utils import (initRandomArchitecture, 
                        initGaussianWeights, initUniformWeights, 
                        dataPath, classify_activity_attractors, save_df)

In [2]:
module # Check location

<module 'binary_model' from 'C:\\Users\\Manu\\Documents\\GitHub\\binary_model_design\\model/__init__.py'>

In [6]:
# The experiment
sim = 'free_evolution'
distribution_weight = 'normal'
experiment = 'dynamic_statistics'
experiment = experiment + '_' + distribution_weight

Ns = [1000, 10000] # Reservoir size. Values tested in the paper: [100, 1000, 10000]
duration = 2000
nb_trial = 10 # Number of different reservoir per sigma value. Values tested in the paper: 100
Ks = [4, 8, 16] # Connectivity degree of the graph. Values tested in the paper: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
meanWeights = [-0.1, 0.1] # Average weights for the normal distribution. Values tested in the paper: [-0.1, 0.1]
I = 0.2 # Fraction of up neural state (20%) at times t=0

# List of standard deviation sigma (of the normal distribution)
nb_sigmas = 5
sigma_0 = 0.001
sigma_end = 100
powers = np.linspace(np.log10(sigma_0), np.log10(sigma_end), nb_sigmas)
sigmas = np.power(10, powers)

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

The model will be evaluated on 5 values of sigma(W):
sigma(W) = [1.00000000e-03 1.77827941e-02 3.16227766e-01 5.62341325e+00
 1.00000000e+02]


In [4]:
# Create the model
def create_rbn_model(N, K, seedConnectivity=747, spikeCount=True, indexActive=False):
    optionData = {'spikeCount':spikeCount, 'indexActive':indexActive} # Option for saving data
    brain = binaryModel(N, sim=sim, experiment=experiment)
    initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture
    return brain

In [5]:
# Function to check if reservoir is ran already

def check_if_ran(sim, experiment, N, K, duration, nbTrial, meanWeight, sigma):
    #Check network has not been run already
    pathBrain = dataPath(sim, experiment, N, K, meanWeight)
    file_path = pathBrain + f"/metadata_N{N}_K{K}_D{duration}_T{nbTrial}_W{meanWeight}_std{sigma}.npy"
    if os.path.exists(file_path):
        print('Network already run, next...\n \n')
        return True
    else:
        return False

In [6]:
# Run the experiment

for N in Ns:
    for K in Ks:
        brain = create_rbn_model(N, K) # Create the RBN architecture
        for meanWeight in meanWeights:
            for sigma in tqdm(sigmas):

                sigma = np.round(sigma, 4)
                optionData = {'spikeCount':True, 'indexActive':False}

                # Check if the network is already run
                check = check_if_ran(sim, experiment, N, K, duration, nb_trial,meanWeight, sigma)
                if check:
                    continue

                print('-------------------')
                print('| Selected network')
                print('|'+ f'N:{N}, K:{K}, sigma:{sigma}')
                print('-------------------')

                for t in range(nb_trial):

                    seed_res = t*100

                    # Create weights
                    if distribution_weight == 'normal':
                        initGaussianWeights(brain, meanWeight, sigma, seed=seed_res)
                    elif distribution_weight == 'uniform':
                        initUniformWeights(brain, meanWeight, sigma, seed=seed_res)

                    # Run simulation
                    brain.displayInfoNetwork()
                    brain.initRandomState(I, seed=seed_res) # Initialize the state of the reservoir
                    brain.run(duration, **optionData)

                    # Reset the network state
                    brain.reset()

                # Save data and metadata
                brain.saveData(**optionData)
                brain.saveMetadata()
                brain.reset(deep=True)


  0%|                                                                  | 0/5 [00:00<?, ?it/s]

-------------------
| Selected network
|N:1000, K:4, sigma:0.001
-------------------
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.001
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.001
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.001
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.001
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.001
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.001
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degr

 20%|███████████▌                                              | 1/5 [00:01<00:06,  1.63s/it]

SAVE C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K4/mu-0.1/std0.001
-------------------
| Selected network
|N:1000, K:4, sigma:0.0178
-------------------
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.0178
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.0178
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.0178
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.0178
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degree K : 4
Mean weights : -0.1
Std weights : 0.0178
---- RUN ! ----
---- Network informations ----
Number of neurons N : 1000
Connectivity degre

 40%|███████████████████████▏                                  | 2/5 [00:03<00:04,  1.65s/it]

...

SAVE C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K16/mu0.1/std0.3162
-------------------
| Selected network
|N:10000, K:16, sigma:5.6234
-------------------
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 5.6234
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 5.6234
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 5.6234
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 5.6234
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 5.6234
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connecti

 80%|██████████████████████████████████████████████▍           | 4/5 [01:20<00:20, 20.59s/it]

SAVE C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K16/mu0.1/std5.6234
-------------------
| Selected network
|N:10000, K:16, sigma:100.0
-------------------
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 100.0
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 100.0
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 100.0
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 100.0
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : 0.1
Std weights : 100.0
---- RUN ! ----
---- Network informations ----
Number of neurons N : 10000
Connectivity d

100%|██████████████████████████████████████████████████████████| 5/5 [01:39<00:00, 19.85s/it]

SAVE C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K16/mu0.1/std100.0


In [16]:
# Function to classify attractor of reservoir's activities
import glob 

def classify_attractor_activity(N, K, nbTrial, meanWeight, duration=2000, plotActivity=False,
                                sim='free_evolution', experiment='dynamic_statistics'):
    
    # Saving folder
    results_dir = dataPath(sim, experiment, N, K, meanWeight)

    # Load filenames
    file = f'/metadata_N{N}_K{K}_D{duration}_T{nbTrial}_W{meanWeight}_*.npy'
    file_list = glob.glob(results_dir +  file)
        
    for filename in file_list:

        # Load informations
        metadata = np.load(filename, allow_pickle='TRUE').item()
        directory = metadata['dataPath']
        fileData = metadata['spikeCountFile']
        stdWeight = metadata.get('stdWeights', None)
        seeds = metadata['seed']
        seed = seeds['seedWeights']
        
        print(filename)
        print('Network seed:', seed)

        # Create folder to save plot
        if plotActivity:
            result_dir = dataPath(sim, experiment, N, K, meanWeights=meanWeight, stdWeights=stdWeight)
            folderTest = 'plots/'
            folder_plot = os.path.join(result_dir, folderTest)
            if not os.path.isdir(folder_plot):
                os.makedirs(folder_plot)
            
        # Load activity data
        print(directory + '/' + fileData + '.npy')
        try:
            A_trials_orig = np.load(directory + '/' + fileData + '.npy')
        except:
            print('--- NOT LOADED:', fileData)
            continue

        intermediateTimeStep = int(duration-1000)
        # Classify all trials
        idxTrial_Periodic = []
        idxTrial_Const = []
        idxTrial_NonStat = []
        idxTrial_Dead = []
        idxTrial_Irregular = []
        activityTrialIrregular = []
        for j in range(nbTrial):

            # Classify the activity into attractor category
            A_trial = np.array(A_trials_orig[j])
            A_trial = A_trial[intermediateTimeStep+1:]
            label = classify_activity_attractors(A_trial, N)
            
            print(label)

            if label == "Cyclic attractor":
                idxTrial_Periodic.append(j)
            elif label == "Fixed point attractor":
                idxTrial_Const.append(j)
            elif label == "No activity":
                idxTrial_Dead.append(j)
            elif label == "Irregular attractor":
                idxTrial_Irregular.append(j)
                activityTrialIrregular.append(A_trial)
            elif label == "Non trivial activity":
                idxTrial_NonStat.append(j)

            if plotActivity:
                # Figure for activity
                fig1 = plt.figure()
                ax1 = fig1.add_subplot(1, 1, 1)
                print(f'Trial #{j}')
                ax1.set_ylabel('A')
                ax1.set_xlabel('TimeStep')
                ax1.plot(A_trial, label=label)
                ax1.legend()
                sig_star = np.round(stdWeight/meanWeight, 3)
                ax1.set_title('$\sigma^{\star}$' + f'={sig_star}, seed={100*j}')
                plt.tight_layout()
                fig1.savefig(folder_plot + f'{label}-activity_trial#{j}_N{N}_K{K}_W{meanWeight}_std{stdWeight}.jpg')
                plt.close()
                
        # Store trials sorted for each sigma(W)
        statAttractors = {'cyclic':len(idxTrial_Periodic)/nbTrial, 
                          'dead':len(idxTrial_Dead)/nbTrial,
                          'nontrivial':len(idxTrial_NonStat)/nbTrial,
                          'irregular':len(idxTrial_Irregular)/nbTrial,
                          'fix':len(idxTrial_Const)/nbTrial,
                          }
        print(statAttractors)
        all_stats_attractors = {'idxPeriodic': idxTrial_Periodic,
                                'idxDead': idxTrial_Dead,
                                'nonStat': idxTrial_NonStat,
                                'idxConst': idxTrial_Const,
                                'idxIrregular': idxTrial_Irregular,
                                'statAttractors':statAttractors,
                                }
        metadata.update(all_stats_attractors)
        print('SAVE metadata at:', filename)
        np.save(filename, metadata)
        print('---------------')

def format_attractor_name(attractor_name):
    dic = {'irregular':'Irregular',
           'dead':'No-Activity',
           'fix':'Fix',
           'cyclic':'Cyclic',
           'nontrivial':'Non-Trivial'}
    return dic.get(attractor_name, None)

# Convert all data into a pandas dataframe for simplfying analysis
def save_stat_attractor_csv(N, K, nbTrial, meanWeight, duration,  
                           sim='free_evolution', 
                           experiment='dynamic_statistics'):
    # Load filenames
    results_dir = dataPath(sim, experiment, N, K, meanWeight)
    filename = f'/metadata_N{N}_K{K}_D{duration}_T{nbTrial}_W{meanWeight}_*.npy'
    file_list = glob.glob(results_dir +  filename)
    
    # Folder for csv
    result_dir = dataPath(sim, experiment)
    commonFolder = os.path.join(result_dir, 'analysis/')
    if not os.path.isdir(commonFolder):
        os.makedirs(commonFolder)
    
    dic_to_csv = {'stdWeights':[],
                 'Irregular':[],
                 'Non-Trivial':[],
                 'Cyclic':[],
                 'No-Activity':[],
                 'Fix':[]} 
    for i, filename in enumerate(file_list):
        # Load informations
        metadata = np.load(filename, allow_pickle='TRUE').item()
        sigma = metadata.get('stdWeights', None)
        stat_attractors = metadata['statAttractors']
        
        # Update dictionary
        dic_to_csv['stdWeights'].append(sigma)
        for attractor, stat in stat_attractors.items():
            dic_to_csv[format_attractor_name(attractor)].append(stat)
    
    
    df_path = commonFolder + f'/statistic_attractors_N{N}_K{K}_W{meanWeight}_d{duration}_T{nbTrial}'
    print(df_path)
    save_df(df_path, dic_to_csv, save=True)
      



In [4]:
# Function to compute Bi-Entropy and stor it in a dataframe
from bientropy import tbien
from bitstring import Bits

def binarize_to_str(continuous_signal, threshold=0.5):
    continuous_signal = np.array(continuous_signal)
    continuous_signal = continuous_signal - min(continuous_signal)
    if max(continuous_signal) != 0:
        continuous_signal = continuous_signal / max(continuous_signal)
    continuous_signal[continuous_signal >= 0.5] = 1
    continuous_signal[continuous_signal < 0.5] = 0
    continuous_signal = np.array(continuous_signal, dtype=int)
    str_list = list(np.array(continuous_signal, dtype=str))
    return ''.join(str_list)

def entropy_versus_sigma(N, K, nbTrial, meanWeight, duration=2000, plot=False,
                        sim='free_evolution', experiment='dynamic_statistics'):
    
    # Saving folder
    results_dir = dataPath(sim, experiment)
    
    # Saving results_dir for all K analysis
    if plot:
        plot_folder = os.path.join(results_dir, 'plots/')
        if not os.path.isdir(plot_folder):
            os.makedirs(plot_folder)
    
    result_dir = dataPath(sim, experiment)
    commonFolder = os.path.join(result_dir, 'analysis/')
    if not os.path.isdir(commonFolder):
        os.makedirs(commonFolder)
    
    # Init figure
    if plot:
        fig = plt.figure()
        ax = fig.add_subplot(211)
        ax.set_title('Average')
        # ax.set_title('Average over trials of the bi-entropy of activity signals')
        ax.set_ylabel('$<BiH(A)>_T$')
        ax1 = fig.add_subplot(212)
        ax1.set_title('Variance')
        # ax1.set_title('Variance over trials of the bi-entropy of activity signals')
        ax1.set_ylabel('$\sigma^2_T(BiH(A))$')
        ax1.set_xlabel('|$\sigma^{\star}|$')
        fig.suptitle(f'$K={K}$')
    
    for i, meanWeight in enumerate(meanWeights):
        # Load filenames
        results_dir = dataPath(sim, experiment, N, K, meanWeights=meanWeight)
        print(results_dir)
        filenames = glob.glob(
            results_dir +  '*metadata*_N' + str(N) + '*_T' + str(nbTrial) + '*_W' + str(meanWeights) + '*.npy')

        avg_entropy = []
        var_entropy = []
        sigmas = []
        for filename in filenames:
            # Load metadata
            metadata = np.load(filename, allow_pickle='TRUE').item()
            duration = metadata['duration']
            sigma = metadata['stdWeights']
            sigmas.append(sigma)
            # Load activity
            directoryActivity = metadata['dataPath']
            fileData = metadata['spikeCountFile']
            idxStart = int(duration / 2)
            print(directoryActivity + fileData)
            A_trials = np.load(directoryActivity + '/' + fileData + '.npy')
            H = []
            for A in A_trials:
                # Apply threshold
                A = A[idxStart:]
                A_bin = binarize_to_str(A, 0.5)
                H.append(tbien(Bits('0b'+A_bin)))
            avg_entropy.append(np.mean(H))
            var_entropy.append(np.var(H))
        
        # Plot
        sigmas = np.array(sigmas)
        avg_entropy = np.array(avg_entropy)
        var_entropy = np.array(var_entropy)
        idxOrder = np.argsort(sigmas)
        if i==0:
            marker = '*-'
        else:
            marker = '.-'            
        if meanWeight > 0:
            label = '$\sigma^{\star}>0$'
        else:
            label = '$\sigma^{\star}<0$'
            
        # Save
        dic_result = {'stdWeights':sigmas[idxOrder],
                      '<BH>':avg_entropy,
                      'var(BH)':var_entropy}
        df_path = commonFolder + f'dataframe_activity_statistics_N{N}_K{K}_W{meanWeight}_d{duration}_T{nbTrial}'
        save_df(df_path, dic_result)
         
        if plot:
            ax.semilogx(sigmas[idxOrder]/abs(meanWeight), avg_entropy[idxOrder], marker, label=label)
            color = ax.get_lines()[-1].get_color()
            ax1.semilogx(sigmas[idxOrder]/abs(meanWeight), var_entropy[idxOrder], marker, color=color)
    
    if plot:
        fig.legend()
        name = f'entropy_over_sigma_N{N}_K{K}_W{meanWeight}_d{duration}_T{nbTrial}'                                  
        fig.savefig(plot_folder+name+'.svg')
        plt.show()


In [17]:
# Compute all dynamic analysis

for N in Ns:
    for K in Ks:
        print('##################')
        print('Compute Bi-Entropy')
        print('##################')
        entropy_versus_sigma(N, K, nb_trial, meanWeights, 
                            duration, plot=False,
                            sim=sim, experiment=experiment)
        
        for meanWeight in meanWeights:
            print('###################')
            print('Classify attractors')
            print('###################')
            classify_attractor_activity(N, K, nb_trial, meanWeight, 
                                        duration, plotActivity=False,
                                        sim=sim, experiment=experiment)
            print('############################')
            print('Compute attractor statistics')
            print('############################')
            save_stat_attractor_csv(N, K, nb_trial, meanWeight, duration,  
                               sim=sim, experiment=experiment)
            

##################
Compute Bi-Entropy
##################
C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal/N1000_K4/mu-0.1/
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K4/mu-0.1/std0.001spikeCount_N1000_K4_D2000_T10_W-0.1_std0.001
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K4/mu-0.1/std0.0178spikeCount_N1000_K4_D2000_T10_W-0.1_std0.0178
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K4/mu-0.1/std0.3162spikeCount_N1000_K4_D2000_T10_W-0.1_std0.3162
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K4/mu-0.1/std100.0spikeCount_N1000_K4_D2000_T10_W-0.1_std100.0
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K4/mu-0.1/std5.6234spikeC

C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K8/mu0.1/std100.0spikeCount_N1000_K8_D2000_T10_W0.1_std100.0
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K8/mu0.1/std5.6234spikeCount_N1000_K8_D2000_T10_W0.1_std5.6234
###################
Classify attractors
###################
C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal/N1000_K8/mu-0.1\metadata_N1000_K8_D2000_T10_W-0.1_std0.001.npy
Network seed: [100, 200, 300, 400, 500, 600, 700, 800, 900]
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K8/mu-0.1/std0.001/spikeCount_N1000_K8_D2000_T10_W-0.1_std0.001.npy
No activity
No activity
No activity
No activity
No activity
No activity
No activity
No activity
No activity
No activity
{'cyclic': 0.0, 'dead': 1.0, 'nontrivial': 0.0, 'irregular': 0.0, 'fix': 0.0}


C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K16/mu-0.1/std5.6234spikeCount_N1000_K16_D2000_T10_W-0.1_std5.6234
C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal/N1000_K16/mu0.1/
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K16/mu0.1/std0.001spikeCount_N1000_K16_D2000_T10_W0.1_std0.001
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K16/mu0.1/std0.0178spikeCount_N1000_K16_D2000_T10_W0.1_std0.0178
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K16/mu0.1/std0.3162spikeCount_N1000_K16_D2000_T10_W0.1_std0.3162
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N1000_K16/mu0.1/std100.0spikeCount_N1000_K16_D2000_T10_W0.1_std100.0
C:\\Users\Manu\

C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal\analysis//statistic_attractors_N1000_K16_W0.1_d2000_T10
Oops! <class 'FileNotFoundError'> occured.
>> [Errno 2] No such file or directory: 'C:\\Users\\Manu\\Documents\\GitHub\\binary_model_design\\results/free_evolution/dynamic_statistics_normal\\analysis//statistic_attractors_N1000_K16_W0.1_d2000_T10.csv'
##################
Compute Bi-Entropy
##################
C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal/N10000_K4/mu-0.1/
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K4/mu-0.1/std0.001spikeCount_N10000_K4_D2000_T10_W-0.1_std0.001
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K4/mu-0.1/std0.0178spikeCount_N10000_K4_D2000_T10_W-0.1_std0.0178
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_e

C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal\analysis//statistic_attractors_N10000_K4_W0.1_d2000_T10
Oops! <class 'FileNotFoundError'> occured.
>> [Errno 2] No such file or directory: 'C:\\Users\\Manu\\Documents\\GitHub\\binary_model_design\\results/free_evolution/dynamic_statistics_normal\\analysis//statistic_attractors_N10000_K4_W0.1_d2000_T10.csv'
##################
Compute Bi-Entropy
##################
C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal/N10000_K8/mu-0.1/
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K8/mu-0.1/std0.001spikeCount_N10000_K8_D2000_T10_W-0.1_std0.001
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_evolution\dynamic_statistics_normal\/N10000_K8/mu-0.1/std0.0178spikeCount_N10000_K8_D2000_T10_W-0.1_std0.0178
C:\\Users\Manu\Documents\GitHub\binary_model_design\/results/free_e

Irregular attractor
Irregular attractor
Irregular attractor
Irregular attractor
Irregular attractor
Irregular attractor
Irregular attractor
{'cyclic': 0.0, 'dead': 0.0, 'nontrivial': 0.0, 'irregular': 1.0, 'fix': 0.0}
SAVE metadata at: C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal/N10000_K8/mu0.1\metadata_N10000_K8_D2000_T10_W0.1_std5.6234.npy
---------------
############################
Compute attractor statistics
############################
C:\Users\Manu\Documents\GitHub\binary_model_design\results/free_evolution/dynamic_statistics_normal\analysis//statistic_attractors_N10000_K8_W0.1_d2000_T10
Oops! <class 'FileNotFoundError'> occured.
>> [Errno 2] No such file or directory: 'C:\\Users\\Manu\\Documents\\GitHub\\binary_model_design\\results/free_evolution/dynamic_statistics_normal\\analysis//statistic_attractors_N10000_K8_W0.1_d2000_T10.csv'
##################
Compute Bi-Entropy
##################
C:\Users\Manu\Documents\GitHub\b